In [30]:
from langchain_community.embeddings import OllamaEmbeddings
#from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
import json
from langchain.prompts import PromptTemplate
from langchain.document_loaders import WebBaseLoader
from langchain_text_splitters import CharacterTextSplitter
import ollama
from bs4 import BeautifulSoup
import requests
from langchain.docstore.document import Document

In [5]:
Embedding_Model = "nomic-embed-text"

In [13]:
docLoader = WebBaseLoader("https://en.wikipedia.org/wiki/Angling")
document = docLoader.load()
text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=500)
docs = text_splitter.split_documents(document)
db = FAISS.from_documents(docs, OllamaEmbeddings(model = Embedding_Model))
retriever = db.as_retriever() 

In [ ]:
url = 'https://en.wikipedia.org/wiki/1896_Summer_Olympics'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [17]:
def combineDocs(docs):
    context = "\n\n".join(f'Document Content : \n{doc.page_content} ]' for doc in docs)
    return context

In [18]:
history = [' ', ' ', ' ', ' ', ' ', ' '] 
def chat(question) : 
    
    global history

    contextString = combineDocs(retriever.invoke(question))
    systemMessage = """ 
    You are a helpful assistant. You are given conversation history and a question with some context from the user. 
    Answer the question based on the information given in that context or based on the conversation history if needed."
    
    """
    message=[
                {
                    'role': 'system',
                    'content': systemMessage,
                },
                {
                    'role': 'user',
                    'content': history[-6],
                },
                {
                    'role': 'assistant',
                    'content': history[-5],
                },
                {
                    'role': 'user',
                    'content': history[-4],
                },
                {
                    'role': 'assistant',
                    'content': history[-3],
                },
                {
                    'role': 'user',
                    'content': history[-2],
                },
                {
                    'role': 'assistant',
                    'content': history[-1],
                },
                {
                    'role': 'user',
                    'content': "Here is the context : \n" + contextString,
                },{
                    'role': 'user',
                    'content': question,
                },
        ]
    
    response = ollama.chat(model = "Gemma2", messages = message)
    answer = response['message']['content']
    history.append(question)
    history.append(answer)
    
    print(answer)
    

In [19]:
chat("Tell me about a fishing rod")

A fishing rod is a long pole used to catch fish. It typically consists of a flexible shaft made of materials like fiberglass, graphite, or bamboo, with a line attached to one end and a reel at the other. Anglers cast the line into the water, wait for a fish to bite, and then reel it in using the rod and reel. 

There are many different types of fishing rods designed for specific types of fish and fishing techniques. Some popular types include:

* **Spinning rods:** Versatile and easy to use, suitable for a variety of freshwater fish.
* **Casting rods:** Powerful rods designed for larger fish and heavier lures.
* **Fly rods:** Used for fly fishing, requiring specialized casting techniques and flies. 


Let me know if you have any other questions about fishing rods or fishing in general! 



In [20]:
chat("what is a bite indicator?")

Please provide me with the conversation history so I can give you an accurate answer about a bite indicator.  

I need context to understand what kind of "bite indicator" you're asking about. For example, is it related to:

* **Dog training?** 
* **Medical devices?**
* **Food packaging?**
* **Something else entirely?**


Let me know, and I'll do my best to help! 😊 



In [2]:
url = 'https://en.wikipedia.org/wiki/1896_Summer_Olympics'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [28]:
lol = soup.findAll("div", class_ = 'mw-body-content')[0].text

AttributeError: 'tuple' object has no attribute 'page_content'